In [3]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [4]:
dataset = pd.read_json('../Input/dataset.json')
dataset.head()

,category,headline,id,short_description
0,crime,There Were 2 Mass Shootings In Texas Last Week...,1,She left her husband. He killed their children...
1,entertainment,Will Smith Joins Diplo And Nicky Jam For The 2...,2,Of course it has a song.
2,entertainment,Hugh Grant Marries For The First Time At Age 57,3,The actor and his longtime girlfriend Anna Ebe...
3,entertainment,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,4,The actor gives Dems an ass-kicking for not fi...
4,entertainment,Julianna Margulies Uses Donald Trump Poop Bags...,5,"The ""Dietland"" actress said using the bags is ..."


In [5]:
# Cleaning the texts
dataset = dataset.replace('\d+', '', regex=True)

In [47]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(min_df=5, encoding='latin-1', ngram_range=(1, 2), stop_words='english', binary=True)
features = vectorizer.fit_transform(dataset['headline'] + ' ' + dataset['short_description'])
features.shape

(200853, 85996)

In [48]:
vectorizer.get_feature_names()

['___',
 'aa',
 'aaa',
 'aaliyah',
 'aap',
 'aapi',
 'aaron',
 'aaron carter',
 'aaron hernandez',
 'aaron paul',
 'aaron persky',
 'aaron rodgers',
 'aaron schock',
 'aaron sorkin',
 'aaron swartz',
 'aarp',
 'ab',
 'ab workout',
 'aback',
 'abandon',
 'abandoned',
 'abandoned building',
 'abandoned buildings',
 'abandoned places',
 'abandoning',
 'abandonment',
 'abandons',
 'abba',
 'abbas',
 'abbey',
 'abbey season',
 'abbi',
 'abbi jacobson',
 'abbott',
 'abbreviated',
 'abby',
 'abby wambach',
 'abc',
 'abc family',
 'abc news',
 'abcs',
 'abdel',
 'abdel fattah',
 'abdeslam',
 'abdicated',
 'abdication',
 'abdomen',
 'abdominal',
 'abducted',
 'abduction',
 'abdul',
 'abdul jabbar',
 'abdulaziz',
 'abdullah',
 'abe',
 'abedin',
 'abercrombie',
 'abercrombie fitch',
 'abhorrent',
 'abide',
 'abiding',
 'abigail',
 'abilities',
 'ability',
 'ability change',
 'ability cope',
 'ability create',
 'ability focus',
 'ability function',
 'ability inspire',
 'ability learn',
 'ability l

In [49]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, dataset['category'], test_size = 0.20, random_state = 0)

In [50]:
# Fitting classifier to the Training set
from sklearn.linear_model import SGDClassifier
sgd_clf = SGDClassifier(early_stopping=True, random_state=0)
sgd_clf.fit(X_train, y_train)

/home/anjaneyaubuntu/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=True, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=0, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)

In [51]:
# Predicting the Test set results
y_pred = sgd_clf.predict(X_test)

In [52]:
n_correct = sum(y_pred == y_test)
print(n_correct / len(y_pred))

0.6020512309875283


In [53]:
from xgboost import XGBClassifier
XGB_clf = XGBClassifier(max_depth=1, n_estimators=10, n_jobs=4)
XGB_clf.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=1, min_child_weight=1, missing=None, n_estimators=10,
       n_jobs=4, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [54]:
# Predicting the Test set results
y_pred_XGB = XGB_clf.predict(X_test)

In [55]:
n_correct_XGB = sum(y_pred_XGB == y_test)
print(n_correct_XGB / len(y_pred_XGB))

0.3312837619178014


In [43]:
from sklearn.model_selection import cross_val_score
cross_val_score(XGB_clf, X_train, y_train, cv=3, scoring='accuracy')

array([0.32685631, 0.32402965, 0.32370903])

In [56]:
from sklearn.ensemble import RandomForestClassifier
forest_clf = RandomForestClassifier(random_state=0)
forest_clf.fit(X_train, y_train)

/home/anjaneyaubuntu/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [59]:
# Predicting the Test set results
y_pred_forest = forest_clf.predict(X_test)

In [60]:
n_correct_forest = sum(y_pred_forest == y_test)
print(n_correct_forest / len(y_pred_forest))

0.48963182395260263


In [62]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
DecisionTreeClassifier(), n_estimators=500,
max_samples=100000, bootstrap=True, n_jobs=-1)
bag_clf.fit(X_train, y_train)
y_pred_bag = bag_clf.predict(X_test)

KeyboardInterrupt: 

In [63]:
from sklearn.naive_bayes import MultinomialNB
MNB_clf = MultinomialNB()
MNB_clf.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [64]:
# Predicting the Test set results
y_pred_MNB = MNB_clf.predict(X_test)

In [66]:
n_correct_MNB = sum(y_pred_MNB == y_test)
print(n_correct_MNB / len(y_pred_MNB))

0.5759378656244555


In [68]:
from sklearn.linear_model import LogisticRegression
LR_clf = LogisticRegression()
LR_clf.fit(X_train, y_train)

/home/anjaneyaubuntu/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/anjaneyaubuntu/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [69]:
# Predicting the Test set results
y_pred_MNB = LR_clf.predict(X_test)

In [70]:
n_correct_MNB = sum(y_pred_MNB == y_test)
print(n_correct_MNB / len(y_pred_MNB))

0.6149212118194718
